In [ ]:
pip install -q langchain youtube-transcript-api gradio tiktoken faiss-cpu openai

import requests
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi

# 🔑 Paste your Gemini API key here
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Gemini API URL (for gemini-2.0-flash)
GEMINI_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}"

# ✂️ Helper to chunk text
def chunk_text(text, max_chars=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_chars, len(text))
        chunks.append(text[start:end])
        start += max_chars - overlap
    return chunks

# 🧠 Call Gemini to summarize
def gemini_summarize(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "contents": [
            {"role": "user", "parts": [{"text": prompt}]}
        ]
    }
    response = requests.post(GEMINI_URL, headers=headers, json=data)
    if response.status_code == 200:
        try:
            return response.json()['candidates'][0]['content']['parts'][0]['text']
        except:
            return "⚠️ Error parsing response"
    else:
        return f"❌ API Error: {response.status_code} - {response.text}"

# 🔁 Main summarizer
def summarize_youtube_video(video_url):
    try:
        # 1. Extract transcript
        video_id = video_url.split("v=")[-1].split("&")[0]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([entry["text"] for entry in transcript])

        # 2. Chunk text
        chunks = chunk_text(full_text)

        # 3. Summarize each chunk with Gemini
        partial_summaries = []
        for i, chunk in enumerate(chunks):
            summary = gemini_summarize(f"Summarize this video transcript segment:\n\n{chunk}")
            partial_summaries.append(summary)

        # 4. Final summary from all chunks
        final_input = "Combine and summarize the following partial summaries:\n\n" + "\n\n".join(partial_summaries)
        final_summary = gemini_summarize(final_input)
        return final_summary

    except Exception as e:
        return f"❌ Error: {str(e)}"

# 🎛️ Gradio UI
gr.Interface(
    fn=summarize_youtube_video,
    inputs=gr.Textbox(label="YouTube Video URL"),
    outputs=gr.Textbox(label="Summary", lines=10),
    title="🎥 YouTube Summarizer with Gemini API",
    description="Paste a YouTube video URL with captions to generate a summary using Gemini 2.0 Flash."
).launch()
